# Data Generation (Dynamic Factor Stochastic Volatility)

We simulate an $N$-dimensional vector of asset returns $ \mathbf{r}_t $ with a **dynamic factor** structure that allows for stochastic volatility in both the common factors and (optionally) the idiosyncratic terms. Let $K$ be the number of latent factors.

---

## 1. Observation Equation

$$
\mathbf{r}_t = \Lambda\,\mathbf{f}_t + \boldsymbol{\epsilon}_t,
$$

- $ \mathbf{r}_t \in \mathbb{R}^N $ is the vector of returns at time $t$.
- $ \Lambda \in \mathbb{R}^{N \times K} $ is the matrix of factor loadings.
- $ \mathbf{f}_t \in \mathbb{R}^K $ is the vector of latent factors capturing the common movements in returns.
- $ \boldsymbol{\epsilon}_t \in \mathbb{R}^N $ is the asset-specific noise term.

We assume $ \boldsymbol{\epsilon}_t $ follows a normal distribution with diagonal covariance:

$$
\boldsymbol{\epsilon}_t \sim \mathcal{N}\bigl(\mathbf{0},\,\Sigma_{\epsilon,t}\bigr),
\quad
\Sigma_{\epsilon,t} = \mathrm{diag}\bigl(e^{h_{(id),1,t}},\,\dots,\,e^{h_{(id),N,t}}\bigr).
$$

In the **baseline** model, $h_{(id),i,t}$ can be held constant over time. However, one can also let each $h_{(id),i,t}$ evolve via an AR(1) process for time-varying idiosyncratic volatility.

---

## 2. Factor Evolution

Each factor $ f_{k,t} $ follows a vector autoregression with time-varying volatility:

$$
\mathbf{f}_{t+1}
= \Phi_f\,\mathbf{f}_t + \boldsymbol{\nu}_{t+1},
\quad
\boldsymbol{\nu}_{t+1} \sim \mathcal{N}\!\Bigl(
\mathbf{0},\,\mathrm{diag}\bigl(e^{h_{1,t+1}},\dots,e^{h_{K,t+1}}\bigr)
\Bigr),
$$

- $ \Phi_f \in \mathbb{R}^{K \times K} $ is the coefficient matrix capturing persistence and cross-factor interactions.
- The diagonal covariance of $ \boldsymbol{\nu}_{t+1} $ depends on factor log-volatilities $ h_{k,t+1} $.

---

## 3. Factor Volatility Dynamics

The factor log-volatility $ \bm h_{t} $ follows an VAR(1) process:

$$
\bm h_{t+1} 
= \bm\mu_
+ \bm\phi_k\bigl(h_{t} - \bm\mu_k\bigr)
+ \bm \eta_{t+1},
\quad
\bm \eta_{t+1} \sim \mathcal{N}(\bm 0,\,\bm Q_h).
$$

- $ \bm \mu_k $ is the long-run mean of the $k$th factor’s log-volatility.
- $ \bm \phi_k $ is the persistence parameter, typically with $|\phi_k| < 1$.
- $ Q_h $ is the covariance matrix of the factor-volatility shocks.

---

## 4. Idiosyncratic Volatility (Optional AR(1))

If modeling **time-varying** idiosyncratic volatility, each $ h_{(id),i,t} $ also evolves via AR(1):

$$
h_{(id),i,t+1}
= \mu_{(id),i}
+ \phi_{(id),i}\bigl(h_{(id),i,t} - \mu_{(id),i}\bigr)
+ \eta_{(id),i,t+1},
\quad
\eta_{(id),i,t+1}
\sim \mathcal{N}\bigl(0,\,\sigma^2_{(id),i}\bigr).
$$

In practice, many factor models keep $ h_{(id),i,t} $ fixed for simplicity, attributing most time variation to the common factors instead.

---

## 5. Generating Returns Step-by-Step

1. **Simulate Factor Volatilities**: For each factor $k$, generate $ h_{k,t} $ via its AR(1) process.
2. **Draw Factor Innovations**: Using $ h_{k,t} $, form the covariance for $ \boldsymbol{\nu}_t $ and draw factor shocks.
3. **Update Factors**: Apply the VAR(1) recursion
   $$
   \mathbf{f}_t = \Phi_f\,\mathbf{f}_{t-1} + \boldsymbol{\nu}_t.
   $$
4. **(Optional) Idiosyncratic Volatility**: If modeling time-varying idiosyncratic risk, update $ h_{(id),i,t} $ for each asset $i$.
5. **Generate Idiosyncratic Noise**: Draw $ \boldsymbol{\epsilon}_t $ from $ \mathcal{N}\!\bigl(\mathbf{0},\,\Sigma_{\epsilon,t}\bigr). $
6. **Compute Returns**: Combine factors and idiosyncratic noise:
   $$
   \mathbf{r}_t = \Lambda\,\mathbf{f}_t + \boldsymbol{\epsilon}_t.
   $$

This **dynamic factor stochastic volatility** framework models time-varying correlations primarily through the shared factors whose volatilities evolve over time, rather than specifying each correlation pairwise. It aligns with the model specification in the thesis proposal, where factor loadings capture common movements across assets and the AR(1) log-volatility processes reflect persistent changes in risk.


# Simulation Steps
1. Pick simulation parameters

In [1]:
import numpy as np
import polars as pl
from sim_functions import DFSV_params
N=2 #number of assets
K=1 #number of factors
T=100 #number of time periods

#mean vector for log-volatilities
#state transition matrix

In [22]:
N, K = 5, 2
lambda_r = np.random.rand(N, K)
Phi_f = np.random.rand(K, K)
Phi_h = np.random.rand(K, K)
mu = np.random.rand(K, 1)
sigma2 = np.random.rand(N, 1)
# Add Q_f as a K×K covariance matrix for factor volatility shocks
Q_h = np.eye(K) * 0.1  # Simple diagonal matrix with 0.1 on the diagonal
DFSV_params(N=N, K=K, lambda_r=lambda_r, Phi_f=Phi_f, 
            Phi_h=Phi_h, mu=mu, sigma2=sigma2, Q_h=Q_h)

DFSV_params(N=5, K=2, lambda_r=array([[0.34474258, 0.30935815],
       [0.78018594, 0.60240245],
       [0.28730041, 0.3624047 ],
       [0.08415374, 0.39147094],
       [0.36956806, 0.55246141]]), Phi_f=array([[0.97565794, 0.12841026],
       [0.46830597, 0.51371548]]), Phi_h=array([[0.19468451, 0.55025439],
       [0.04194431, 0.35781702]]), mu=array([[0.47900234],
       [0.23673526]]), sigma2=array([[0.82045575],
       [0.04250422],
       [0.90989945],
       [0.69425972],
       [0.38384071]]), Q_h=array([[0.1, 0. ],
       [0. , 0.1]]))

In [23]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import inv
from numpy.linalg import slogdet

def get_pair_indices(N):
    """Return list of pairs (i, j) with i < j for N assets."""
    pairs = []
    for i in range(N):
        for j in range(i+1, N):
            pairs.append((i, j))
    return pairs

def construct_sigma(x, N):
    """
    Given state vector x (length d = N + M), where:
      - x[0:N] = h (latent log-volatilities),
      - x[N:] = theta (latent parameters for correlations),
    construct the covariance matrix Sigma = D R D.
    """
    h = x[:N]
    M = int(N*(N-1)/2)
    theta = x[N: N+M]
    
    # Construct diagonal matrix D = diag(sqrt(exp(h)))
    D = np.diag(np.sqrt(np.exp(h)))
    
    # Build correlation matrix R from latent theta values.
    R = np.eye(N)
    pairs = get_pair_indices(N)
    for idx, (i, j) in enumerate(pairs):
        rho = np.tanh(theta[idx])
        R[i, j] = rho
        R[j, i] = rho
    # Full covariance:
    Sigma = D @ R @ D
    return Sigma

def log_likelihood(x, r, N):
    """
    Compute the log-likelihood ln p(r | x) where
    r ~ N(0, Sigma(x)).
    """
    Sigma = construct_sigma(x, N)
    sign, logdet = slogdet(Sigma)
    if sign <= 0:
        # Return a very low log-likelihood if Sigma is not positive definite.
        return -np.inf
    ll = -0.5 * (logdet + r.T @ inv(Sigma) @ r)
    return ll

def objective(x, r, x_pred, P_pred, N):
    """
    The negative log-posterior (to be minimized):
    -[ ln p(r|x) + ln p(x|x_pred) ]
    where p(x|x_pred) is Gaussian with mean x_pred and covariance P_pred.
    """
    # Observation likelihood (assumed Gaussian):
    ll = log_likelihood(x, r, N)
    if ll == -np.inf:
        return 1e10  # Penalize non-positive definite Sigma heavily
    
    # Prior term: -0.5 * (x - x_pred)' P_pred^{-1} (x - x_pred)
    diff = x - x_pred
    prior = -0.5 * diff.T @ inv(P_pred) @ diff
    
    # Negative log-posterior
    return - (ll + prior)

def filter_update(x_pred, P_pred, r, N, x0=None):
    """
    Given a predicted state x_pred and covariance P_pred, and an observation r,
    update the state estimate by maximizing the posterior likelihood using likelihood optimization.
    x is the state vector of length d = N + M, where M = N*(N-1)/2.
    """
    d = N + int(N*(N-1)/2)
    if x0 is None:
        x0 = x_pred.copy()  # Use predicted state as initial guess
    
    res = minimize(objective, x0, args=(r, x_pred, P_pred, N), method='BFGS')
    if not res.success:
        print("Optimization failed at this time step:", res.message)
    x_updated = res.x
    # Estimate updated covariance as the inverse Hessian at optimum
    P_updated = res.hess_inv
    return x_updated, P_updated

# Example usage for a single time-step update:
if __name__ == '__main__':
    # Model dimensions
    N = 3  # number of assets
    M = int(N*(N-1)/2)
    d = N + M  # total state dimension
    
    # True state (for simulation) – here we set it arbitrarily
    true_h = np.array([0.0, 0.2, -0.1])
    true_theta = np.array([0.1, -0.2, 0.05])  # three unique pairs for N=3
    x_true = np.concatenate([true_h, true_theta])
    
    # Simulated observed return vector (for one time step)
    # For example, we simulate using the true state
    Sigma_true = construct_sigma(x_true, N)
    r_obs = np.random.multivariate_normal(np.zeros(N), Sigma_true)
    
    # Suppose we have a predicted state and covariance (from previous time step prediction)
    # Here we set them arbitrarily as well.
    x_pred = x_true + np.random.normal(0, 0.05, size=d)
    P_pred = np.diag(0.05*np.ones(d))
    
    # Update state using our filter
    x_updated, P_updated = filter_update(x_pred, P_pred, r_obs, N)
    
    # For clarity, separate the updated h and theta:
    h_updated = x_updated[:N]
    theta_updated = x_updated[N:]
    
    print("Predicted state (x_pred):\n", x_pred)
    print("\nTrue state (x_true):\n", x_true)
    print("\nObserved return (r_obs):\n", r_obs)
    print("\nUpdated state (x_updated):\n", x_updated)
    print("\nUpdated latent log-volatilities (h_updated):\n", h_updated)
    print("\nUpdated latent correlation parameters (theta_updated):\n", theta_updated)


Predicted state (x_pred):
 [-0.07096863  0.25341717 -0.07854114  0.09058538 -0.14220153  0.00763581]

True state (x_true):
 [ 0.    0.2  -0.1   0.1  -0.2   0.05]

Observed return (r_obs):
 [-1.07991737 -0.13884561  1.92678411]

Updated state (x_updated):
 [-0.07533459  0.2285987  -0.01674661  0.09787565 -0.21574113  0.00368936]

Updated latent log-volatilities (h_updated):
 [-0.07533459  0.2285987  -0.01674661]

Updated latent correlation parameters (theta_updated):
 [ 0.09787565 -0.21574113  0.00368936]


In [1]:
import jax
jax.devices()

[CpuDevice(id=0)]